In [1]:
import numpy as np
import scipy.stats as sps
import scipy.integrate as spi

In [12]:
table = [
    [83, 85],
    [84, 85, 85, 86, 86, 87],
    [86, 87, 87, 87, 88, 88, 88, 88, 88, 89, 90],
    [89, 90, 90, 91],
    [90, 92]
]
n = 25
p = 4

In [4]:
sample = []
for i, sample_ig in enumerate(table):
    for _ in range(len(sample_ig)):
        sample.append(np.eye(5)[i])

sample = np.array(sample)

In [5]:
Y = [83, 85, 84, 85, 85, 86, 86, 87, 86, 87, 87, 87, 88, 88, 88, 88, 88, 89, 90, 89, 90, 90, 91, 90, 92]

In [6]:
F = sample.T @ sample
F_inv = np.linalg.inv(F)
B = F_inv @ sample.T @ Y
e = Y - sample @ B
RSS = e.T @ e
TSS = np.sum((Y - np.average(Y)) ** 2)
R_2 = (TSS - RSS) / TSS

In [7]:
print(f'Коэффициенты регрессии: {B}')
print(f'RSS регрессии: {RSS}')
print(f'TSS регрессии: {TSS}')
print(f'R^2 регрессии: {R_2}')

Коэффициенты регрессии: [84.         85.5        87.81818182 90.         91.        ]
RSS регрессии: 23.136363636363633
TSS регрессии: 122.16
R^2 регрессии: 0.8106060606060607


In [13]:
delta = R_2 / (1 - R_2) * (n - p) / (p - 1)
p_value = spi.quad(sps.f.pdf, delta, np.inf, args=(p - 1, n - p))[0]
print(f'p_value: {p_value}')
if p_value < 0.05:
    print('Регрессия значима')
else:
    print('Регрессия не значима')

p_value: 8.895768529347218e-08
Регрессия значима


In [16]:
deltas = [[] for _ in range(p - 1)]
p_values = [[] for _ in range(p - 1)]

for i in range(0, p - 1):
    for j in range(i + 1, p):
        b1 = B[i]
        b2 = B[j]
        
        F_sum = F_inv[i][i] + F_inv[j][j]
        
        delta = (b1 - b2) / np.sqrt(RSS * F_sum) * np.sqrt(n - p)
        p_value = sps.t.cdf(delta, n - p)
        deltas[i].append(delta)
        p_values[i].append(p_value)

In [17]:
def flat(array):
    result = []
    for element in array:
        for el in element:
            result.append(el)
    
    return result

In [18]:
p_values = flat(p_values)
deltas = flat(deltas)

In [20]:
p_values = np.sort(p_values)
for i, p_value in enumerate(p_values, 0):
    if p_value > 0.05 / (10 - i):
        print(i, p_value, 0.05 / (10 - i))

5 0.047337163681430906 0.01
